In [1]:
import json
import pandas as pd
from pathlib import Path
import re

# --- Configuration ---
# This script assumes it is run from a directory containing the dataset folders.
ROOT_DIR = Path('.') 
MODELS = ['llama_3_3_70b_instruct', 'llama_4']
SEEDS = [42, 101, 1234, 2024, 12345]
DATASETS = ['dailylifeapis', 'huggingface']

# --- MODIFICATION: Updated method mapping and added SPIRAL ---
ABLATION_METHODS_MAP = {
    'Baseline MCTS (Light)': 'light',
    'Baseline MCTS (Medium)': 'medium',
    'Baseline MCTS (Heavy)': 'heavy',
    'Greedy (w/o Plan History)': 'no_mcts',
    'SPIRAL w/o Simulator': 'no_sim_feedback',
    'SPIRAL w/o Validator': 'no_validator',
    'SPIRAL w/ Uniform Rewards': 'uniform_rewards',
    'SPIRAL (Full)': 'spiral' 
}

# --- Data Parsing ---
all_results = []

print("Starting analysis for RQ3 (Ablation Study)...")

for dataset in DATASETS:
    print(f"\nProcessing Dataset: {dataset.upper()}")
    
    for model in MODELS:
        print(f"  Processing Model: {model}")

        for method_name, method_folder in ABLATION_METHODS_MAP.items():
            
            for seed in SEEDS:
                try:
                    # Construct the path based on the corrected folder structure
                    summary_path = ROOT_DIR / dataset / model / method_folder / f'run_seed_{seed}' / 'summary.json'

                    if not summary_path.exists():
                        raise StopIteration # Handle missing files gracefully

                    with open(summary_path, 'r') as f:
                        summary = json.load(f)
                        accuracy = float(summary['final_accuracy'].strip('%'))
                        
                        all_results.append({
                            'dataset': dataset,
                            'model': model,
                            'method': method_name,
                            'seed': seed,
                            'accuracy': accuracy
                        })

                except StopIteration:
                    print(f"    - ⚠️ WARNING: Could not find summary for {dataset}/{model}/{method_folder}, seed {seed}. Skipping.")
                    continue
                except Exception as e:
                    print(f"    - 🔴 ERROR: Parsing failed for {dataset}/{model}/{method_folder}, seed {seed} -> {e}")
                    continue

# --- Aggregate and Print Table ---
if all_results:
    df = pd.DataFrame(all_results)
    
    # Calculate mean and std across seeds
    summary_table = df.groupby(['dataset', 'model', 'method'])['accuracy'].agg(['mean', 'std']).reset_index()
    
    # Format for display
    summary_table['std'] = summary_table['std'].fillna(0)
    summary_table['Final Accuracy'] = summary_table.apply(
        lambda row: f"{row['mean']:.2f}% ± {row['std']:.2f}", axis=1
    )
    
    # Reorder methods for logical presentation
    method_order = list(ABLATION_METHODS_MAP.keys())
    summary_table['method'] = pd.Categorical(summary_table['method'], categories=method_order, ordered=True)
    summary_table = summary_table.sort_values(['dataset', 'model', 'method'])

    # Pivot for final presentation
    final_table = summary_table.pivot_table(
        index=['model', 'method'],
        columns='dataset',
        values='Final Accuracy',
        aggfunc='first'
    )
    
    print("\n" + "="*80)
    print("📊 RQ3: Ablation Study on SPIRAL Components (Overall Accuracy)")
    print("="*80)
    print(final_table.to_string())
    print("\n")

else:
    print("🔴 No results were generated. Please check the file paths and error messages.")



Starting analysis for RQ3 (Ablation Study)...

Processing Dataset: DAILYLIFEAPIS
  Processing Model: llama_3_3_70b_instruct
  Processing Model: llama_4

Processing Dataset: HUGGINGFACE
  Processing Model: llama_3_3_70b_instruct
  Processing Model: llama_4

📊 RQ3: Ablation Study on SPIRAL Components (Overall Accuracy)
dataset                                           dailylifeapis    huggingface
model                  method                                                 
llama_3_3_70b_instruct Baseline MCTS (Light)      87.60% ± 1.31  89.68% ± 1.43
                       Baseline MCTS (Medium)     87.11% ± 1.72  89.12% ± 0.95
                       Baseline MCTS (Heavy)      86.28% ± 1.71  89.44% ± 1.03
                       Greedy (w/o Plan History)  94.71% ± 1.90  87.48% ± 1.42
                       SPIRAL w/o Simulator       92.89% ± 1.11  69.48% ± 3.02
                       SPIRAL w/o Validator       87.27% ± 2.38  90.08% ± 1.30
                       SPIRAL w/ Uniform Rewards 

/tmp/ipykernel_1831368/136290419.py:84: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  final_table = summary_table.pivot_table(
